In [1]:
import nltk
import re
import math
import string
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [2]:
#states = pd.read_csv('state abbr.csv', encoding = 'utf-8')

In [3]:
data = pd.read_csv('Corona_NLP_train.csv', encoding = 'utf-8')

In [4]:
def clean_text(text):
    # Make text lowercase, remove text in square brackets, remove links, remove punctuation and remove words containing numbers
    # Use re.sub to substitude all that with ''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)                              # remove text in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text)                # remove links
    text = re.sub('<.*?>+', '', text)                               # remove text in angle brackets
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('\n', '', text)                                   # remove line break
    text = re.sub('\w*\d\w*', '', text)                             # remove words containing numbers
    return(text)

In [5]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = re.sub(emoji_pattern, '', text)
    return(text)

In [6]:
data['text'] = data['OriginalTweet'].apply(lambda x: clean_text(x))

In [7]:
data['text'] = data['text'].apply(lambda x: remove_emoji(x))

# LDA for all the tweets

In [8]:

from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [9]:

stop_words = stopwords.words('english')

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data['text']))

In [11]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'
])

id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]] 
#it will print the words with their frequencies.
lda_model = gensim.models.ldamodel.LdaModel(
   corpus=corpus, id2word=id2word, num_topics=5, random_state=100, 
   update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True
)
pprint(lda_model.print_topics())


[(0,
  '0.083*"price" + 0.045*"food" + 0.025*"oil" + 0.025*"demand" + 0.018*"need" '
  '+ 0.017*"due" + 0.014*"buy" + 0.014*"market" + 0.014*"stock" + '
  '0.013*"increase"'),
 (1,
  '0.039*"amp" + 0.023*"help" + 0.023*"time" + 0.020*"new" + 0.019*"crisis" + '
  '0.017*"good" + 0.017*"use" + 0.016*"keep" + 0.013*"still" + 0.013*"covid"'),
 (2,
  '0.040*"online" + 0.031*"worker" + 0.029*"shopping" + 0.020*"business" + '
  '0.018*"shop" + 0.017*"high" + 0.015*"customer" + 0.014*"delivery" + '
  '0.012*"support" + 0.012*"order"'),
 (3,
  '0.100*"consumer" + 0.048*"pandemic" + 0.023*"impact" + 0.017*"company" + '
  '0.014*"sanitizer" + 0.013*"behavior" + 0.013*"show" + 0.013*"economy" + '
  '0.012*"hand" + 0.010*"sale"'),
 (4,
  '0.064*"coronavirus" + 0.038*"store" + 0.035*"grocery" + 0.033*"supermarket" '
  '+ 0.028*"go" + 0.026*"people" + 0.019*"get" + 0.016*"see" + 0.016*"make" + '
  '0.015*"work"')]


In [14]:
    for topic_id in range(5):
        print('Topic#%d：\t' % topic_id)
        term_distribute_all = lda_model.get_topic_terms(topicid=topic_id)
        term_distribute = term_distribute_all[:5]
        term_distribute = np.array(term_distribute)
        term_id = term_distribute[:, 0].astype(np.int)
        print('Words：\t',)
        for t in term_id:
            print(id2word.id2token[t],)
        print('\nProb：\t', term_distribute[:, 1])

Topic#0：	
Words：	
price
food
oil
demand
need

Prob：	 [0.08315818 0.04487643 0.02505431 0.02488246 0.01811617]
Topic#1：	
Words：	
amp
help
time
new
crisis

Prob：	 [0.03858849 0.0234146  0.02314532 0.01958633 0.0185634 ]
Topic#2：	
Words：	
online
worker
shopping
business
shop

Prob：	 [0.03963828 0.03146422 0.02889453 0.02009588 0.01834049]
Topic#3：	
Words：	
consumer
pandemic
impact
company
sanitizer

Prob：	 [0.09952403 0.04770662 0.02297371 0.01746472 0.01354481]
Topic#4：	
Words：	
coronavirus
store
grocery
supermarket
go

Prob：	 [0.0642093  0.03803531 0.03483826 0.03271723 0.0278498 ]


In [15]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# how good the model is. The lower the score the better the model will be

coherence_model_lda = CoherenceModel(
   model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#the average /median of the pairwise word-similarity scores of the words in the topic




Perplexity:  -7.849486445552428

Coherence Score:  0.31383536527474076


# LDA for each labeled tweets

In [16]:
data_positive=data['text'][data['Sentiment']=='Positive']


In [17]:
data_negative=data['text'][data['Sentiment']=='Negative']

data_EP=data['text'][data['Sentiment']=='Extremely Positive']

data_EN=data['text'][data['Sentiment']=='Extremely Negative']
data_neutral=data['text'][data['Sentiment']=='Neutral']

In [18]:
data_neutral=data_neutral.drop([0])

In [19]:
def get_topics(df):
    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    data_words = list(sent_to_words(df))
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc))
                 if word not in stop_words] for doc in texts]
    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]
    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
            return texts_out
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops)
    nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'])
    id2word = corpora.Dictionary(data_lemmatized)
    texts = data_lemmatized

    corpus =[id2word.doc2bow(text) for text in texts] 
    [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]] 
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=1, random_state=100, 
    update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    pprint(lda_model.print_topics())
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))
    coherence_model_lda = CoherenceModel(
    model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)


    


In [20]:
topic_positive=get_topics(data_positive.values)


[(0,
  '0.059*"phone" + 0.059*"number" + 0.059*"neighbour" + 0.039*"supply" + '
  '0.039*"poss" + 0.039*"online" + 0.039*"order" + 0.039*"shopping" + '
  '0.039*"regular" + 0.039*"set"')]

Perplexity:  -3.510801986411766

Coherence Score:  0.9999999999999998


In [21]:
topic_negative=get_topics(data_negative.values)

[(0,
  '0.091*"online" + 0.061*"prefer" + 0.061*"time" + 0.061*"thing" + '
  '0.061*"stop" + 0.061*"spread" + 0.061*"shopping" + 0.061*"also" + '
  '0.061*"payment" + 0.061*"home"')]

Perplexity:  -3.115619389449849

Coherence Score:  1.0


In [22]:
topic_EP=get_topics(data_EP.values)

[(0,
  '0.086*"time" + 0.057*"particularly" + 0.057*"pack" + 0.057*"thank" + '
  '0.057*"situation" + 0.057*"share" + 0.057*"patience" + 0.057*"wait" + '
  '0.057*"due" + 0.057*"may"')]

Perplexity:  -3.1786561409632363

Coherence Score:  1.0


In [23]:
topic_EN=get_topics(data_EN.values)

[(0,
  '0.053*"confinement" + 0.053*"stayathome" + 0.053*"serious" + '
  '0.053*"restezchezvous" + 0.053*"ready" + 0.053*"panic" + '
  '0.053*"supermarket" + 0.053*"stock" + 0.053*"shortage" + 0.053*"be"')]

Perplexity:  -3.3123766319616776

Coherence Score:  1.0


In [24]:
#data_neutral=data['text'][data['Sentiment']=='Neutral']
topic_neutral=get_topics(data_neutral.values)

[(0, '0.250*"buy" + 0.250*"rebel" + 0.250*"supermarket" + 0.250*"today"')]

Perplexity:  -1.683350554824898

Coherence Score:  1.0


# LSI Model for similarity

In [25]:
object = pd.read_pickle(r'week_words.pickle')


In [26]:
data_weekly = object.values()

In [56]:
data_weekly = list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is not None, data_weekly)) 

<>:1: DeprecationWarning: invalid escape sequence \s
<>:1: DeprecationWarning: invalid escape sequence \s
<ipython-input-56-0af2f55d90fe>:1: DeprecationWarning: invalid escape sequence \s
  data_weekly = list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is not None, data_weekly))


In [58]:
data_po="".join(data_positive)

In [59]:
data_ne="".join(data_negative)

In [60]:
data_ep="".join(data_EP)

In [61]:
data_en="".join(data_EN)

In [62]:
from gensim import models,similarities

In [204]:
def simi(s1,s2):
    dictionary= corpora.Dictionary([s1.split()])
    texts = s1

    corpus =[dictionary.doc2bow(text) for text in [s1.split()]] 

    lsi=models.LdaModel(corpus, id2word=dictionary, num_topics=4)

    vec_bow=dictionary.doc2bow(s2.lower().split())
    vec_lsi=lsi[vec_bow]  #convert the query to LSI space

     
    #transform corpus to space and index it
    index=similarities.MatrixSimilarity(lsi[corpus])
     

    sims=index[vec_lsi]#?
    sims=sorted(enumerate(sims),key=lambda item:-item[1])
     
  
    print(np.mean(sims))
  
    

In [205]:
simi(data_po,data_weekly[0])

0.48889344930648804


In [206]:
simi(data_ne,data_weekly[3])

0.49993452429771423


In [207]:
simi(data_en,data_weekly[0])

0.49908721446990967


In [208]:
simi(data_ep,data_weekly[2])

0.47187837958335876


In [224]:
len(data_weekly)

29

In [223]:
list = [data_po,data_ne,data_ep, data_en]

for i in range(len(list)):
        for j in range(len(data_weekly)):
            try:
                dt=simi(list[i], data_weekly[j])
            except:
                try:
                    print(i,j)
                    dt=simi(list[i], data_weekly[j])
                except:
                    print(i,j)
                    dt=simi(list[i], data_weekly[j])
            #dt=simi(list[i], data_weekly[j])
            dt=pd.DataFrame(dt)
            dt.to_csv('dt.csv',index=False)
               

0.49930670857429504
0.4967852830886841
0.4999352991580963
0.47077518701553345
0.499090313911438
0.48993760347366333
0.4995453953742981
0.4983910620212555
0.48360931873321533
0.4998330771923065
0.49934902787208557
0.4994356036186218
0.4998909533023834
0.4998686909675598
0.4742996096611023
0.4893410801887512
0.49919113516807556
0.4777284860610962
0.4988222122192383
0.4811556041240692
0.4994298219680786
0.49996238946914673
0.49725213646888733
0.49999603629112244
0.5
0.49837130308151245
0.49297085404396057
0.49610435962677
0.4975164830684662
0.49300384521484375
0.48386019468307495
0.48928865790367126
1 3
0.48490503430366516
0.4975071847438812
0.48314717411994934
0.4964180588722229
0.4969451427459717
0.49959152936935425
0.49926841259002686
0.49974578619003296
0.49993696808815
0.4642929136753082
0.4953835606575012
0.4999728798866272
0.49311941862106323
0.49942702054977417
0.49621981382369995
0.4970610439777374
0.4955645203590393
0.49993985891342163
0.4999181926250458
0.49998658895492554
1 23